In [55]:
import numpy as np
import os
import pandas as pd
from osmgraph import GraphFromBbox, GraphFromHmlFile, GraphFromGdfs
from spaitalShape import Point, OdPair, Box
from edgeGdfPreprocessing import edgePreprocessing
import osmnx as ox
import math
import time

In [2]:
class LocationRequest:
    def __init__(self):
        # Murphy Depot
        # (longitude, latitude)
        self.origin = Point(-93.2219, 44.979)
        # test
        #self.origin = Point(-93.4254, 44.7888)
        #self.origin = Point(-93.470167, 44.799720)
        #origin = Point(-93.2466, 44.8959)
        # Shakopee East (Depot):
        self.destination = Point(-93.4620, 44.7903)
        #self.destination = Point(-93.230358, 44.973583)

        self.odPair = OdPair(self.origin, self.destination)
        self.boundingBox = Box(-93.4975, -93.1850, 44.7458, 45.0045)
        self.mass = 23000
        # 9 am
        self.timeOfTheDay = 9
        # Monday
        self.dayOfTheWeek = 1

In [4]:
def extractGraphOf(boundingBox):
    folderOfGraph = r'GraphDataInBbox'
    if os.path.exists(folderOfGraph):
        print("reloading graph..")
        osmGraph = GraphFromHmlFile(os.path.join(folderOfGraph, 'osmGraph.graphml'))
    else:
        print("downloading graph..")
        osmGraph = GraphFromBbox(boundingBox)
        osmGraph.saveHmlTo(folderOfGraph)
    # fig, ax = ox.plot_graph(osmGraph.graph)
    return osmGraph


def extractElevation(nodes, edges):
    extractNodesElevation(nodes)
    extractEdgesElevation(nodes, edges)



def extractNodesElevation(nodes):
    nodesElevation = pd.read_csv("nodesWithElevation.csv", index_col=0)
    nodes['indexId'] = nodes.index
    nodes['elevation'] = nodes.apply(lambda x: nodesElevation.loc[x['indexId'], 'MeanElevation'], axis=1)



def extractEdgesElevation(nodesWithElevation, edges):
    edges['uElevation'] = edges['u'].apply(lambda x: nodesWithElevation.loc[x,'elevation'])
    edges['vElevation'] = edges['v'].apply(lambda x: nodesWithElevation.loc[x,'elevation'])


def extractGraphInMurphy(nodes, edges):
    edgesInMurphy = extractEdgesInMurphy(edges)
    graphInMurphy = GraphFromGdfs(nodes, edgesInMurphy)
    return graphInMurphy


def extractEdgesInMurphy(edges):
    edges['uvPair'] = edges.apply(lambda x: (x.u, x.v), axis=1)
    segmentElevationChange = np.load('statistical data/segmentElevationChange.npy', allow_pickle=True).item()
    edges['isInMurphy'] = edges.uvPair.apply(lambda x: x in segmentElevationChange)
    return edges[edges.isInMurphy]


def findShortestPath(osmGraph, localRequest):
    shortestPath = osmGraph.shortestPath(localRequest)
    print("shortestPath:", shortestPath)
    # ox.plot_graph(osmGraph)
    #osmGraph.plotPath(shortestPath, "shortest route.pdf")
    return shortestPath


def findEcoPathAndCalEnergy(osmGraph, localRequest):
    ecoPath, ecoEnergy , ecoEdgePath = osmGraph.ecoPath(localRequest)
    print("ecoPath:", ecoPath, "ecoEnergy:", ecoEnergy, ecoEdgePath)
    #osmGraph.plotPath(ecoPath, "eco route.pdf")
    return ecoPath, ecoEnergy,  ecoEdgePath


def findFastestPathAndCalTime(osmGraph, localRequest):

    fastestPath, shortestTime, fastEdgePath = osmGraph.fastestPath(localRequest)
    print("fastestPath:", fastestPath, "shortestTime:", shortestTime, fastEdgePath)
    #osmGraph.plotPath(fastestPath,"fastest route.pdf")
    return fastestPath, shortestTime, fastEdgePath


def nodePathTOEdgePath(nodePath, edgesGdf):
    edgePath = []
    for i, OdPair in enumerate(zip(nodePath[:-1], nodePath[1:])):
        segmentId = edgesGdf[edgesGdf['odPair'] == OdPair].index[0]
        edgePath.append(segmentId)
    return edgePath



def calAndPrintPathAttributes(osmGraph, edgePath, pathname):
    numberOfSegments = len(edgePath)
    length = osmGraph.totalLength(edgePath)
    energy = osmGraph.totalEnergy(edgePath)
    time = osmGraph.totalTime(edgePath)
    print(pathname+":"+f"{numberOfSegments} segments, {length} meters, {energy} liters, {time} seconds")
    return

In [58]:
import numpy as np
import os
import pandas as pd
from osmgraph import GraphFromBbox, GraphFromHmlFile, GraphFromGdfs
from spaitalShape import Point, OdPair, Box
from edgeGdfPreprocessing import edgePreprocessing
import osmnx as ox
import math
import time

In [59]:
t0 = time.time()
locationRequest = LocationRequest()
osmGraphInBbox = extractGraphOf(locationRequest.boundingBox)
nodes, edges = osmGraphInBbox.graphToGdfs()
extractElevation(nodes, edges)

reloading graph..


In [60]:
edges = edgePreprocessing(nodes, edges, locationRequest)

In [ ]:
graphWithElevation = GraphFromGdfs(nodes, edges)
graphWithElevation.removeIsolateNodes()

In [47]:
from window import Window
w = Window(61851,61877,61893,33971)

In [39]:
edgesGdf = graphWithElevation.getEdges()

In [57]:
edges.loc[0]

osmid                                                             6022209
name                                                     East 43rd Street
highway                                                       residential
maxspeed                                                           30 mph
oneway                                                              False
length                                                            100.067
geometry                LINESTRING (-93.265072 44.9251429, -93.2649186...
ref                                                                   NaN
lanes                                                                   0
bridge                                                                NaN
tunnel                                                                NaN
access                                                                NaN
junction                                                              NaN
area                                  

In [41]:
w.extractFeatures(edgesGdf)

([[-1.4862292198533746,
   -0.022488209690081074,
   -0.031192437596675903,
   0.49512859685839516,
   -0.5971237458925717,
   -0.5792325804809063],
  [-1.4862292198533746,
   -0.022488209690081074,
   1.2821756220539557,
   0.9571575011038456,
   -0.2850785784460404,
   -0.018281071138699402],
  [-1.4862292198533746,
   -0.022488209690081074,
   0.0005224968476190941,
   2.778756984652004,
   0.20143502886916562,
   nan]],
 [[2, 3, 1, 0, 0, 0, 0], [11, 3, 1, 0, 0, 0, 0], [0, 3, 1, 0, 0, 0, 0]])

In [13]:
edgesGdf = graphWithElevation.getEdges()